<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>

# Data Formatting (strings and numeric)

Let me clean some data sets:

In [1]:
# links to websites
carbonLink="https://www.cia.gov/the-world-factbook/field/carbon-dioxide-emissions/country-comparison" 
forestLink="https://www.cia.gov/the-world-factbook/field/revenue-from-forest-resources/country-comparison" 

# scrapping into LIST of Dataframes

import pandas as pd

carbonList=pd.read_html(carbonLink,header=0,flavor='bs4')
forestList=pd.read_html(forestLink,header=0,flavor='bs4')

# getting the Dataframe from list
carbon=carbonList[0]
forest=forestList[0]

# no spaces in column names
carbon.columns=carbon.columns.str.replace(r'\s','',regex=True)
forest.columns=forest.columns.str.replace(r'\s','',regex=True)

#renaming
newCarbonNames={'metrictonnesofCO2':'co2_tonnes','DateofInformation':'Carbon_yearData'}
newForestNames={'%ofGDP':'ForestRevenue_PctGDP', 'DateofInformation':'Forest_yearData'}
carbon.rename(columns=newCarbonNames,inplace=True)
forest.rename(columns=newForestNames,inplace=True)

# dropping
toDrop=['Rank']
carbon.drop(columns=toDrop,inplace=True)
forest.drop(columns=toDrop,inplace=True)

# no spaces in string values (to several columns)
byeSpaces=lambda x: x.str.strip()
carbon.iloc[:,[0,2]]=carbon.iloc[:,[0,2]].apply(byeSpaces)
forest.iloc[:,[0,2]]=forest.iloc[:,[0,2]].apply(byeSpaces)

# keeping year
carbon.Carbon_yearData=carbon.Carbon_yearData.str. extract(pat=r'(\d+)')
forest.Forest_yearData=forest.Forest_yearData.str. extract(pat=r'(\d+)')

Verifying year:

In [2]:
forest[forest.Forest_yearData.str.contains(r'\D')]

,Country,ForestRevenue_PctGDP,Forest_yearData


In [3]:
carbon[carbon.Carbon_yearData.str.contains(r'\D')]

,Country,co2_tonnes,Carbon_yearData


**Before** starting formatting we check the data types:

In [4]:
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               204 non-null    object 
 1   ForestRevenue_PctGDP  204 non-null    float64
 2   Forest_yearData       204 non-null    object 
dtypes: float64(1), object(2)
memory usage: 4.9+ KB


In [5]:
carbon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Country          218 non-null    object
 1   co2_tonnes       218 non-null    int64 
 2   Carbon_yearData  218 non-null    object
dtypes: int64(1), object(2)
memory usage: 5.2+ KB


# String case

This is our string column

In [6]:
carbon.Country

0                                              China
1                                      United States
2                                              India
3                                             Russia
4                                              Japan
                           ...                      
213                                       Antarctica
214    Saint Helena, Ascension, and Tristan da Cunha
215                                             Niue
216                         Northern Mariana Islands
217                                           Tuvalu
Name: Country, Length: 218, dtype: object

In [7]:
# do we have duplicates?
carbon[carbon.duplicated(subset='Country')]

,Country,co2_tonnes,Carbon_yearData


In [8]:
# do we have weird symbols?
carbon[carbon.Country.str.contains(r'[^\w\s]')]

,Country,co2_tonnes,Carbon_yearData
6,"Korea, South",686954000,2019
16,Turkey (Turkiye),391792000,2019
86,"Korea, North",18465000,2019
99,Cote d'Ivoire,11880000,2019
137,"Congo, Republic of the",4523000,2019
141,"Bahamas, The",3984000,2019
150,"Congo, Democratic Republic of the",2653000,2019
185,"Gambia, The",606000,2019
188,Timor-Leste,538000,2019
192,Guinea-Bissau,342000,2019


In [9]:
# accents in words:
carbon[carbon.Country.str.contains(r"\w*[\u00C0-\u01DA']\w*")]

,Country,co2_tonnes,Carbon_yearData
99,Cote d'Ivoire,11880000,2019


In [10]:
# only ascii
from unidecode import unidecode

carbon['Country']=carbon.Country.apply(unidecode)
forest['Country']=forest.Country.apply(unidecode)

ModuleNotFoundError: No module named 'unidecode'

The capitalization is an important step, it may help in later stages when merging data frames:

* str.lower(): all to lowercase.

* str.upper(): ALL TO UPPERCASE.

* str.title(): First Character Of Each Word To Uppercase. 

* str.capitalize(): First character to uppercase and remaining to lowercase.

**You can only apply this if the cells are strings.**

Let's do it:

In [ ]:
carbon_test=carbon.copy()
carbon_test['countryname']=carbon_test.Country.str.lower()
carbon_test['COUNTRYNAME']=carbon_test.Country.str.upper()
carbon_test['CountryName']=carbon_test.Country.str.title()
carbon_test

In [11]:
#Let's keep the upper case
carbon['Country']=carbon.Country.str.upper()
forest['Country']=forest.Country.str.upper()

In [ ]:
#el problema del data es

In [12]:
# we can save this:
import os

carbon.to_csv(os.path.join("data","carbon_formatted.csv"),index=False)
forest.to_csv(os.path.join("data","forest_formatted.csv"),index=False)

OSError: Cannot save file into a non-existent directory: 'data'

# Numeric case

If numeric data **looks** good, but the computernhas not recognised it as a number we are still in trouble:

In [ ]:
#looks good
carbon.Carbon_yearData

In [ ]:
# not numeric data
carbon.Carbon_yearData.info()

In [ ]:
# see ONE element
carbon.Carbon_yearData[0]

You can not get any **expected** statistics if the values are not recognised as numeric:

In [ ]:
carbon.Carbon_yearData.describe()

The easiest way is to use the **pd.to_numeric** function in pandas:

In [ ]:
# now you get stats for numeric data
pd.to_numeric(carbon.Carbon_yearData).describe()

Then:

In [ ]:
carbon['Carbon_yearData']=pd.to_numeric(carbon.Carbon_yearData)
carbon.info()

In [ ]:
forest['Forest_yearData']=pd.to_numeric(forest.Forest_yearData)
forest.info()

Let's overwrite our previous files

In [ ]:
carbon.to_csv(os.path.join("data","carbon_formatted.csv"),index=False)
forest.to_csv(os.path.join("data","forest_formatted.csv"),index=False)

Notice that this works only if the column has already been cleaned:

In [ ]:
# a list of values
listValues=[1,2,3,4,'5','x']
dictValues={'someVals':listValues}
# a series
aDataFrame=pd.DataFrame(dictValues)

aDataFrame

In [ ]:
pd.to_numeric(aDataFrame.someVals)

In [ ]:
pd.to_numeric(aDataFrame.someVals,errors='coerce')

In [ ]:
pd.to_numeric(aDataFrame.someVals,errors='ignore')

In this case, the **coerce** argument would be the best choice.